# Video Assembly Walkthrough

This notebook demonstrates the video assembly pipeline that stitches AI-generated images
and TTS audio into an MP4 video with Ken Burns effects and transitions.

## What it does
1. Creates synthetic test data (solid-color images + silent audio)
2. Calls `assemble_video()` to produce an MP4
3. Shows file info and embeds the video for playback

In [1]:
import sys
sys.path.insert(0, '/app')

import os
import struct
import tempfile
from pathlib import Path

from PIL import Image

from src.clients.video import assemble_video

## Create synthetic test data

In production, these would be FLUX-generated images (768x1360) and TTS WAV files.
Here we use solid-color PNGs and silent WAVs.

In [2]:
def make_test_png(path, width=768, height=1360, color=(255, 0, 0)):
    """Create a solid-color PNG."""
    img = Image.new("RGB", (width, height), color)
    img.save(path, "PNG")


def make_silent_wav(path, duration=2.0, sample_rate=44100):
    """Create a silent WAV file."""
    num_samples = int(duration * sample_rate)
    data_size = num_samples * 2
    with open(path, "wb") as f:
        f.write(struct.pack("<4sI4s", b"RIFF", 36 + data_size, b"WAVE"))
        f.write(struct.pack("<4sIHHIIHH", b"fmt ", 16, 1, 1, sample_rate, sample_rate * 2, 2, 16))
        f.write(struct.pack("<4sI", b"data", data_size))
        f.write(b"\x00" * data_size)

In [3]:
# Create temp directory and synthetic segment data
work_dir = tempfile.mkdtemp(prefix="video_assembly_")
print(f"Working directory: {work_dir}")

segment_configs = [
    {"color": (220, 50, 50),  "duration": 3.0, "transition": "cut",        "label": "Hook (red)"},
    {"color": (50, 180, 80),  "duration": 4.0, "transition": "fade",       "label": "Intro (green)"},
    {"color": (50, 80, 220),  "duration": 3.5, "transition": "slide_left", "label": "Features (blue)"},
    {"color": (220, 180, 50), "duration": 3.0, "transition": "zoom_in",    "label": "Audience (yellow)"},
    {"color": (150, 50, 200), "duration": 2.5, "transition": "fade",       "label": "Closing (purple)"},
]

segments = []
for i, cfg in enumerate(segment_configs):
    img_path = os.path.join(work_dir, f"segment_{i}.png")
    wav_path = os.path.join(work_dir, f"segment_{i}.wav")
    make_test_png(img_path, 768, 1360, color=cfg["color"])
    make_silent_wav(wav_path, duration=cfg["duration"])
    segments.append({
        "image_path": img_path,
        "audio_path": wav_path,
        "audio_duration_seconds": cfg["duration"],
        "transition": cfg["transition"],
    })
    print(f"  Segment {i}: {cfg['label']} — {cfg['duration']}s, transition={cfg['transition']}")

print(f"\nCreated {len(segments)} segments")

Working directory: /tmp/video_assembly__uf_wevg
  Segment 0: Hook (red) — 3.0s, transition=cut
  Segment 1: Intro (green) — 4.0s, transition=fade
  Segment 2: Features (blue) — 3.5s, transition=slide_left
  Segment 3: Audience (yellow) — 3.0s, transition=zoom_in
  Segment 4: Closing (purple) — 2.5s, transition=fade

Created 5 segments


## Assemble the video

Using small dimensions (270x480) for fast rendering in this demo.
Production videos use 1080x1920.

In [4]:
output_path = os.path.join(work_dir, "assembled_video.mp4")

duration = assemble_video(
    segments=segments,
    output_path=output_path,
    width=270,
    height=480,
    fps=15,
    transition_duration=0.5,
)

file_size = os.path.getsize(output_path)
print(f"Output: {output_path}")
print(f"Duration: {duration:.1f}s")
print(f"File size: {file_size / 1024:.1f} KB")

Output: /tmp/video_assembly__uf_wevg/assembled_video.mp4
Duration: 14.0s
File size: 19.6 KB


## Embed video for playback (Jupyter only)

In [5]:
try:
    from IPython.display import Video, display
    display(Video(output_path, embed=True, width=270))
except ImportError:
    print(f"IPython not available — view the video at: {output_path}")